<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/SeoulGrandPark_Prediction_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall --y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 7.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 3.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=92dbdfa26e6999c822a9409399b371d89890ea4f3a28b07b1ec4863fdfc6c46a
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 28 06:01 kaggle.json


In [3]:
!kaggle competitions download -c 2020-ai-termproject-18011817

  0% 0.00/10.5k [00:00<?, ?B/s]
100% 10.5k/10.5k [00:00<00:00, 8.63MB/s]


In [7]:
!unzip 2020-ai-termproject-18011817.zip

Archive:  2020-ai-termproject-18011817.zip
  inflating: submit_sample.csv       
  inflating: test_seoul_grandpark.csv  
  inflating: train_seoul_grandpark.csv  


In [8]:
import pandas as pd
import numpy as np
import random

import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import torchvision.datasets as data
import torchvision.transforms as transforms

from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

In [111]:
device = torch.device('cuda')

torch.manual_seed(777)
random.seed(777)
torch.cuda.manual_seed_all(777)

learning_rate = 0.01
training_epochs = 5000
batch_size = 100

In [112]:
xy_train = pd.read_csv('train_seoul_grandpark.csv', header = None, skiprows=1, usecols=range(1, 8))

x_data = xy_train.iloc[: , 1:-1]
y_data = xy_train.iloc[: , [-1]]

x_data = np.array(x_data)
y_data = np.array(y_data)

scaler = MinMaxScaler()
x_data = scaler.fit_transform(x_data)

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

x_train.shape

torch.Size([636, 5])

In [113]:
train_dataset = TensorDataset(x_train, y_train)
data_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size, 
                                           shuffle = True, 
                                           drop_last = True)

In [114]:
class MishFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return x * torch.tanh(F.softplus(x))   # x * tanh(ln(1 + exp(x)))

    @staticmethod
    def backward(ctx, grad_output):
        x = ctx.saved_variables[0]
        sigmoid = torch.sigmoid(x)
        tanh_sp = torch.tanh(F.softplus(x)) 
        return grad_output * (tanh_sp + x * sigmoid * (1 - tanh_sp * tanh_sp))

class Mish(nn.Module):
    def forward(self, x):
        return MishFunction.apply(x)

def to_Mish(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, Mish())
        else:
            to_Mish(child)

In [115]:
linear1 = torch.nn.Linear(5, 8,bias=True)
linear2 = torch.nn.Linear(8, 8,bias=True)
linear3 = torch.nn.Linear(8, 8,bias=True)
linear4 = torch.nn.Linear(8, 8,bias=True)
linear5 = torch.nn.Linear(8, 1,bias=True)
#dropout = torch.nn.Dropout(p=drop_prob)
mish = Mish()

In [116]:
torch.nn.init.kaiming_normal_(linear1.weight)
torch.nn.init.kaiming_normal_(linear2.weight)
torch.nn.init.kaiming_normal_(linear3.weight)
torch.nn.init.kaiming_normal_(linear4.weight)
torch.nn.init.kaiming_normal_(linear5.weight)

model = torch.nn.Sequential(linear1,mish,
                            linear2,mish,
                            linear3,mish,
                            linear4,mish,
                            linear5).to(device)

In [117]:
loss = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

losses = []
model_history = []
err_history = []

total_batch = len(data_loader)

for epoch in range(training_epochs + 1):
  avg_cost = 0
  #model.train()
  
  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = loss(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
    
  model_history.append(model)
  err_history.append(avg_cost)
  
  if epoch % 100 == 0:  
    print('Epoch:', '%d' % (epoch + 1), 'Cost =', '{:.9f}'.format(avg_cost))
  losses.append(cost.item())
print('Learning finished')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  if __name__ == '__main__':


Epoch: 1 Cost = 107983696.000000000
Epoch: 101 Cost = 48140944.000000000
Epoch: 201 Cost = 40604264.000000000
Epoch: 301 Cost = 38773612.000000000
Epoch: 401 Cost = 39574460.000000000
Epoch: 501 Cost = 39615208.000000000
Epoch: 601 Cost = 36950244.000000000
Epoch: 701 Cost = 40165628.000000000
Epoch: 801 Cost = 38875604.000000000
Epoch: 901 Cost = 39880316.000000000
Epoch: 1001 Cost = 40449148.000000000
Epoch: 1101 Cost = 41269708.000000000
Epoch: 1201 Cost = 40066040.000000000
Epoch: 1301 Cost = 40332416.000000000
Epoch: 1401 Cost = 39766912.000000000
Epoch: 1501 Cost = 38269304.000000000
Epoch: 1601 Cost = 39625404.000000000
Epoch: 1701 Cost = 39999212.000000000
Epoch: 1801 Cost = 40118288.000000000
Epoch: 1901 Cost = 39769528.000000000
Epoch: 2001 Cost = 40260092.000000000
Epoch: 2101 Cost = 39938396.000000000
Epoch: 2201 Cost = 40292960.000000000
Epoch: 2301 Cost = 39942452.000000000
Epoch: 2401 Cost = 39867656.000000000
Epoch: 2501 Cost = 37567544.000000000
Epoch: 2601 Cost = 4010

In [118]:
best_model = model_history[np.argmin(err_history)]

In [119]:
xy_test = pd.read_csv('test_seoul_grandpark.csv', header = None, skiprows=1, usecols = range(1, 7))
x_data = xy_test.iloc[:, 1:]
x_data = np.array(x_data)
x_data = scaler.transform(x_data)
x_test = torch.FloatTensor(x_data).to(device)

with torch.no_grad():
    model.eval()     
    predict = best_model(x_test)


In [120]:
submit = pd.read_csv('submit_sample.csv')
submit['Expected'] = submit['Expected'].astype(float)
for i in range(len(predict)):
  submit['Expected'][i] = predict[i]
submit.to_csv('submit.csv', mode = 'w', index = False, header = True)
submit

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Id,Expected
0,20170330,9434.703125
1,20180418,8279.458984
2,20190430,14760.791992
3,20180618,10350.997070
4,20160329,9403.028320
5,20170424,14475.183594
6,20180719,3818.627686
7,20170828,9298.170898
8,20180227,3713.683105
9,20160211,4209.882324


In [121]:
!kaggle competitions submit -c 2020-ai-termproject-18011817 -f submit.csv -m "14010974_이기택"

100% 761/761 [00:03<00:00, 194B/s]
Successfully submitted to SejongAI.텀프로젝트.[서울대공원 입장객 수 예측]